### Creación del Dataset

In [1]:
#Cargue de Librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os

# Configuración de la semilla para reproducibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# ---------------------------
# Generación del dataset simulado
# ---------------------------
start_date = "2024-03-05 00:00:00"

# 3 meses (3*30 días) de datos cada 15 min
periods = int((3 * 30 * 24 * 60) / 15)
freq = "15min"

# Generar rango de fechas
fechas = pd.date_range(start=start_date, periods=periods, freq=freq)

# Parámetros base y umbrales para detección de condicion_anormal
base_temp = 60      # °C
th_temp = 80        # °C para alarma por temperatura alta

base_vib = 30       # Hz
th_vib = 50         # Hz para alarma por vibración alta

base_pres = 100000  # Pa
th_pres_min = 90000 # Pa para alarma por presión baja

base_corr = 10      # A
th_corr = 15        # A para alarma por corriente alta

base_hum = -50       # ºC (valor base de humedad negativa)
th_hum = -46         # ºC para alarma por humedad alta (menos negativa)

th_caudal_min = 3800  # m³/h para alarma por caudal bajo
base_caudal = 4200    # m³/h 

# Inicializar listas para almacenar datos
temperatura = []
vibracion = []
presion = []
corriente = []
humedad = []
caudal = []
alarma = []

# Variables para la degradación acumulada
degradacion_temp = 0
degradacion_vib = 0
degradacion_pres = 0
degradacion_corr = 0

# Factores de degradación (incremento cada 30 minutos)
fact_degr_temp = 0.55   
fact_degr_vib = 0.35
fact_degr_pres = 15     
fact_degr_corr = 0.06

# Variables para eventos ambientales en caudal (lluvia)
evento_lluvia = False
duracion_lluvia = 0

def simula_caudal(base, evento):
    if evento:
        # Durante lluvia: aumento del caudal entre 10% y 30%
        return base * np.random.uniform(1.10, 1.30)
    else:
        # Variación normal ±5%
        return base * np.random.uniform(0.95, 1.05)

# Variables para eventos de caída (dip) en presión y corriente
evento_dip_pres = False
duracion_dip_pres = 0
evento_dip_corr = False
duracion_dip_corr = 0

# Bucle de simulación
for i in range(periods):
    # Acumular degradación en cada paso
    degradacion_temp += fact_degr_temp
    degradacion_vib += fact_degr_vib
    degradacion_pres += fact_degr_pres
    degradacion_corr += fact_degr_corr
    
    # Generar valores base con degradación y ruido aleatorio
    temp_val = base_temp + degradacion_temp + np.random.normal(0, 0.5)
																
    vib_val = base_vib + degradacion_vib + np.random.normal(0, 0.3)
    pres_val = base_pres + degradacion_pres + np.random.normal(0, 50)
    corr_val = base_corr + degradacion_corr + np.random.normal(0, 0.2)
    hum_val = base_hum + np.random.normal(0, 1)
    
    # Evento de lluvia: baja probabilidad de iniciar
    if not evento_lluvia and np.random.rand() < 0.001:
        evento_lluvia = True
        # Duración aleatoria entre 4 y 12 intervalos (2 a 6 horas)
        duracion_lluvia = np.random.randint(4, 12)
    if evento_lluvia:
        duracion_lluvia -= 1
        if duracion_lluvia <= 0:
            evento_lluvia = False

    caudal_val = simula_caudal(base_caudal, evento_lluvia)

    # Correlación directa: caudal afecta temperatura y vibración
    temp_val = base_temp + degradacion_temp + np.random.normal(0, 0.5) + 0.04 * (caudal_val - base_caudal)
    vib_val = base_vib + degradacion_vib + np.random.normal(0, 0.3) + 0.002 * (caudal_val - base_caudal)

    # Correlación inversa: humedad con temperatura
    hum_val = base_hum + np.random.normal(0, 1) - 0.08 * (temp_val - base_temp)
    
    # Evento de caída (dip) para simular condiciones críticas bajas en Presión y Corriente
    # Se activa con baja probabilidad y dura unos pocos intervalos.
    
    # Evento de caída independiente para presión
    if not evento_dip_pres and np.random.rand() < 0.00001:
        evento_dip_pres = True
        duracion_dip_pres = np.random.randint(1, 3)
    if evento_dip_pres:
        dip_presion = np.random.uniform(3000, 5000)
        pres_val -= dip_presion
        duracion_dip_pres -= 1
        if duracion_dip_pres <= 0:
            evento_dip_pres = False

    # Evento de caída independiente para corriente
    if not evento_dip_corr and np.random.rand() < 0.002:
        evento_dip_corr = True
        duracion_dip_corr = np.random.randint(1, 3)
    if evento_dip_corr:
        dip_corriente = np.random.uniform(1, 3)
        corr_val -= dip_corriente
        duracion_dip_corr -= 1
        if duracion_dip_corr <= 0:
            evento_dip_corr = False

    if np.random.rand() < 0.02:  # Aumentar probabilidad de condicion anormal por temperatura alta
        temp_val = th_temp + np.random.uniform(1, 5)
    if np.random.rand() < 0.02:  # Aumentar probabilidad de condicion anormal por vibraciones altas
        vib_val = th_vib + np.random.uniform(1, 3)
    if np.random.rand() < 0.015:  # aumentar probabilidad de condicion anormal por corriente altas
        corr_val = th_corr + np.random.uniform(0.5, 2)																																		  
    # Evento de humedad anómala (condicion anormal aleatorio)
    if np.random.rand() < 0.01:
        hum_val = th_hum + np.random.uniform(0, 2)  # Puede llegar hasta -44
    # Evento de condicion anormal aleatorio de caudal (bajo o alto)
    if np.random.rand() < 0.090:
        if np.random.rand() < 0.5:
            caudal_val = th_caudal_min - np.random.uniform(1, 100)
        else:
            caudal_val = base_caudal + np.random.uniform(1, 100)

    # Si el caudal es muy bajo, incrementar probabilidad de condicion anormal
    # if caudal_val < th_caudal_min + 100:
    #     if np.random.rand() < 0.60:  # 25% de probabilidad extra de condición anormal
    #         alarma = 1

    # Detección de alarmas:
    # Se marca alarma si:
    # - La temperatura supera el umbral superior.
    # - La vibración supera el umbral superior.
    # - La presión es demasiado alta o baja.
    # - La corriente es demasiado alta o baja.
    condicion_anormal = 0
    if (temp_val > th_temp or 
        vib_val > th_vib or 
        pres_val < th_pres_min or 
        corr_val > th_corr or 
        caudal_val < th_caudal_min or
        hum_val > th_hum):
        condicion_anormal = 1

    # Al ocurrir una alarma se simula un mantenimiento, reseteando la degradación.
    if condicion_anormal:
        degradacion_temp = 0
        degradacion_vib = 0
        degradacion_pres = 0
        degradacion_corr = 0

    # Guardar valores simulados
    temperatura.append(temp_val)
    vibracion.append(vib_val)
    presion.append(pres_val)
    corriente.append(corr_val)
    humedad.append(hum_val)
    caudal.append(caudal_val)
    alarma.append(condicion_anormal)

# Crear DataFrame final
df = pd.DataFrame({
    "Date": fechas,
    "Temperature": temperatura,
    "Vibration": vibracion,
    "Pressure": presion,
    "Current": corriente,
    "Humidity": humedad,
    "Caudal": caudal,
    "Unhealthy": alarma
})

# Mostrar las primeras filas
print(df.head())
print("\nResumen de condiciones anormales detectadass:", df["Unhealthy"].sum())

# Guardado de información
df.to_csv("dataset.csv", index=False)

                 Date  Temperature  Vibration       Pressure    Current  \
0 2024-03-05 00:00:00    66.584691  31.131352  100047.384427  10.364606   
1 2024-03-05 00:15:00    59.990772  30.474256  100070.822254   9.815225   
2 2024-03-05 00:30:00    67.997727  31.640593  100014.914669  10.550456   
3 2024-03-05 00:45:00    58.250629  30.876819  100009.545733   9.923341   
4 2024-03-05 01:00:00    57.924326  30.545140  100004.641705   9.935305   

    Humidity       Caudal  Unhealthy  
0 -49.759341  4353.793981          0  
1 -49.401541  4181.549393          0  
2 -51.860662  4371.914569          0  
3 -44.369077  4107.992494          1  
4 -50.732154  4120.612575          0  

Resumen de condiciones anormales detectadass: 1030


In [2]:

# Leer el último registro de dataset.csv
df1 = pd.read_csv('dataset.csv')
last_date = pd.to_datetime(df1['Date'].iloc[-1]) 

# Parámetros de simulación (idénticos a los usados en la celda principal)
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

periods = int((15 * 24 * 60) / 15)  # 15 días, cada 15 minutos
freq = "15min"
fechas = pd.date_range(start=last_date + pd.Timedelta(minutes=15), periods=periods, freq=freq)

base_temp = 60
th_temp = 80
base_vib = 30
th_vib = 50
base_pres = 100000
th_pres_min = 90000
base_corr = 10
th_corr = 15
base_hum = -50
th_hum = -46
th_caudal_min = 3800
base_caudal = 4200

temperatura = []
vibracion = []
presion = []
corriente = []
humedad = []
caudal = []
alarma = []

degradacion_temp = 0
degradacion_vib = 0
degradacion_pres = 0
degradacion_corr = 0

fact_degr_temp = 0.55
fact_degr_vib = 0.35
fact_degr_pres = 15
fact_degr_corr = 0.06

evento_lluvia = False
duracion_lluvia = 0

def simula_caudal(base, evento):
    if evento:
        return base * np.random.uniform(1.10, 1.30)
    else:
        return base * np.random.uniform(0.95, 1.05)

evento_dip_pres = False
duracion_dip_pres = 0
evento_dip_corr = False
duracion_dip_corr = 0

for i in range(periods):
    degradacion_temp += fact_degr_temp
    degradacion_vib += fact_degr_vib
    degradacion_pres += fact_degr_pres
    degradacion_corr += fact_degr_corr

    temp_val = base_temp + degradacion_temp + np.random.normal(0, 0.5)
    vib_val = base_vib + degradacion_vib + np.random.normal(0, 0.3)
    pres_val = base_pres + degradacion_pres + np.random.normal(0, 50)
    corr_val = base_corr + degradacion_corr + np.random.normal(0, 0.2)
    hum_val = base_hum + np.random.normal(0, 1)

    if not evento_lluvia and np.random.rand() < 0.001:
        evento_lluvia = True
        duracion_lluvia = np.random.randint(4, 12)
    if evento_lluvia:
        duracion_lluvia -= 1
        if duracion_lluvia <= 0:
            evento_lluvia = False

    caudal_val = simula_caudal(base_caudal, evento_lluvia)

    temp_val = base_temp + degradacion_temp + np.random.normal(0, 0.5) + 0.04 * (caudal_val - base_caudal)
    vib_val = base_vib + degradacion_vib + np.random.normal(0, 0.3) + 0.002 * (caudal_val - base_caudal)
    hum_val = base_hum + np.random.normal(0, 1) - 0.08 * (temp_val - base_temp)

    if not evento_dip_pres and np.random.rand() < 0.00001:
        evento_dip_pres = True
        duracion_dip_pres = np.random.randint(1, 3)
    if evento_dip_pres:
        dip_presion = np.random.uniform(3000, 5000)
        pres_val -= dip_presion
        duracion_dip_pres -= 1
        if duracion_dip_pres <= 0:
            evento_dip_pres = False

    if not evento_dip_corr and np.random.rand() < 0.002:
        evento_dip_corr = True
        duracion_dip_corr = np.random.randint(1, 3)
    if evento_dip_corr:
        dip_corriente = np.random.uniform(1, 3)
        corr_val -= dip_corriente
        duracion_dip_corr -= 1
        if duracion_dip_corr <= 0:
            evento_dip_corr = False

    if np.random.rand() < 0.02:
        temp_val = th_temp + np.random.uniform(1, 5)
    if np.random.rand() < 0.02:
        vib_val = th_vib + np.random.uniform(1, 3)
    if np.random.rand() < 0.015:
        corr_val = th_corr + np.random.uniform(0.5, 2)
    if np.random.rand() < 0.01:
        hum_val = th_hum + np.random.uniform(0, 2)
    if np.random.rand() < 0.090:
        if np.random.rand() < 0.5:
            caudal_val = th_caudal_min - np.random.uniform(1, 100)
        else:
            caudal_val = base_caudal + np.random.uniform(1, 100)

    condicion_anormal = 0
    if (temp_val > th_temp or 
        vib_val > th_vib or 
        pres_val < th_pres_min or 
        corr_val > th_corr or 
        caudal_val < th_caudal_min or
        hum_val > th_hum):
        condicion_anormal = 1

    if condicion_anormal:
        degradacion_temp = 0
        degradacion_vib = 0
        degradacion_pres = 0
        degradacion_corr = 0

    temperatura.append(temp_val)
    vibracion.append(vib_val)
    presion.append(pres_val)
    corriente.append(corr_val)
    humedad.append(hum_val)
    caudal.append(caudal_val)
    alarma.append(condicion_anormal)

df2 = pd.DataFrame({
    "Date": fechas,
    "Temperature": temperatura,
    "Vibration": vibracion,
    "Pressure": presion,
    "Current": corriente,
    "Humidity": humedad,
    "Caudal": caudal,
    "Unhealthy": alarma
})

print(df2.head())
print("\nResumen de condiciones anormales detectadas en dataset_test:", df2["Unhealthy"].sum())
df2.to_csv("dataset_test.csv", index=False)

                 Date  Temperature  Vibration       Pressure    Current  \
0 2024-06-03 00:00:00    66.584691  31.131352  100047.384427  10.364606   
1 2024-06-03 00:15:00    59.990772  30.474256  100070.822254   9.815225   
2 2024-06-03 00:30:00    67.997727  31.640593  100014.914669  10.550456   
3 2024-06-03 00:45:00    58.250629  30.876819  100009.545733   9.923341   
4 2024-06-03 01:00:00    57.924326  30.545140  100004.641705   9.935305   

    Humidity       Caudal  Unhealthy  
0 -49.759341  4353.793981          0  
1 -49.401541  4181.549393          0  
2 -51.860662  4371.914569          0  
3 -44.369077  4107.992494          1  
4 -50.732154  4120.612575          0  

Resumen de condiciones anormales detectadas en dataset_test: 173
